## Tablice Rozproszone w Języku Julia

### Tablice w Julii 
Co warto wiedzieć?

In [25]:
#Przykładowa tablica 4x4
A=rand(4,4)

4×4 Array{Float64,2}:
 0.442029   0.739513  0.555822   0.783535
 0.918924   0.45684   0.0488534  0.490098
 0.0310092  0.952315  0.0527051  0.35621 
 0.16121    0.262807  0.464051   0.222112

In [2]:
# tablice indeksowane są od 1 !
A[1,1]

0.7592298400127797

In [3]:
# Należy pamiętać o "column-major" dostępie do tablic - 
# pierwszy indeks zmienia się szybciej
# tak jak Matlab, R, Fortran 
# inaczej niz C, Python
vec(A)

16-element Array{Float64,1}:
 0.75923  
 0.525472 
 0.627278 
 0.399921 
 0.765131 
 0.0648613
 0.425914 
 0.501631 
 0.373734 
 0.183923 
 0.525591 
 0.843161 
 0.389722 
 0.791188 
 0.639701 
 0.0202176

In [4]:
first(A)

0.7592298400127797

In [5]:
last(A)

0.020217624377653953

In [6]:
size(A)

(4, 4)

### Tablice rozproszone

https://github.com/JuliaParallel/DistributedArrays.jl

In [8]:
addprocs(5)

5-element Array{Int64,1}:
 2
 3
 4
 5
 6

In [9]:
nworkers()

5

In [10]:
# @everywhere - wykonuje sie na kazdym procesie
@everywhere using DistributedArrays

In [11]:
#tworzymy tablicę rozproszona pomiędzy workery 
DA1 = @DArray [(i, j) for i = 1:5, j = 1:5]

5×5 DistributedArrays.DArray{Tuple{Int64,Int64},2,Array{Tuple{Int64,Int64},2}}:
 (1, 1)  (1, 2)  (1, 3)  (1, 4)  (1, 5)
 (2, 1)  (2, 2)  (2, 3)  (2, 4)  (2, 5)
 (3, 1)  (3, 2)  (3, 3)  (3, 4)  (3, 5)
 (4, 1)  (4, 2)  (4, 3)  (4, 4)  (4, 5)
 (5, 1)  (5, 2)  (5, 3)  (5, 4)  (5, 5)

In [12]:
# localindexes - funkcja pokazujaca jaki zakres indeksów obejmuje dany worker
#funkcja remotecall() zwraca future, które później jest użyte do ściągnięcia wyniku w funkcji fetch() 
refs = [remotecall(localindexes,w,DA1) for w in workers()]
for r in refs
     d = fetch(r)
     println(d)
end

(1:5, 1:1)
(1:5, 2:2)
(1:5, 3:3)
(1:5, 4:4)
(1:5, 5:5)


In [13]:
# localpart - jaka część tablicy przyporządkowana jest danemu workerowi
refs = [remotecall(localpart,w,DA1) for w in workers()]
for r in refs
     d = fetch(r)
     print("----------\n")
     println(d)
end

----------
Tuple{Int64,Int64}[(1, 1); (2, 1); (3, 1); (4, 1); (5, 1)]
----------
Tuple{Int64,Int64}[(1, 2); (2, 2); (3, 2); (4, 2); (5, 2)]
----------
Tuple{Int64,Int64}[(1, 3); (2, 3); (3, 3); (4, 3); (5, 3)]
----------
Tuple{Int64,Int64}[(1, 4); (2, 4); (3, 4); (4, 4); (5, 4)]
----------
Tuple{Int64,Int64}[(1, 5); (2, 5); (3, 5); (4, 5); (5, 5)]


In [14]:
#przykład przetwarzania tablicy rozproszonej
@everywhere function processingExample(d::DArray)
        println(size(d), " ", procs(d))
         # konstruktor tablicy rozproszonej, podajemy całkowity rozmiar, listę procesów oraz blok do...end inicjalizujacy 
         # tablicę
        DArray(size(d),procs(d)) do I
          println("mój zakres ", I)
          fill( myid(),(length(I[1]), length(I[2])))
        end
end

In [15]:
processingExample(DA1)

(5, 5) [2 3 4 5 6]
	From worker 2:	mój zakres (1:5, 1:1)
	From worker 4:	mój zakres (1:5, 3:3)
	From worker 6:	mój zakres (1:5, 5:5)
	From worker 5:	mój zakres (1:5, 4:4)
	From worker 3:	mój zakres (1:5, 2:2)


5×5 DistributedArrays.DArray{Int64,2,Array{Int64,2}}:
 2  3  4  5  6
 2  3  4  5  6
 2  3  4  5  6
 2  3  4  5  6
 2  3  4  5  6

### Gra w życie
https://pl.wikipedia.org/wiki/Gra_w_%C5%BCycie

In [26]:
# Przykład komunikacji z najbliższymi sąsiadami -  gra w życie
#Sciaganie danych od sąsiadów jest realizowane przez kopiowanie danych wg odpowiednich indeksów
@everywhere function life_step(d::DArray)
        println(size(d), " ", procs(d))
         # konstruktor tablicy rozproszonej, podajemy całkowity rozmiar, listę procesów oraz blok do...end inicjalizujacy 
         # tablicę
        DArray(size(d),procs(d)) do I
        
            # granice powiększone o brzegi sąsiadów
            top   = mod(first(I[1])-2,size(d,1))+1
            bot   = mod( last(I[1])  ,size(d,1))+1
            left  = mod(first(I[2])-2,size(d,2))+1
            right = mod( last(I[2])  ,size(d,2))+1
            println("Mój zakres ", I, " mój zakres + sąsiadujące brzegi ", " bot=", bot, " top=",top," left=", left, " right=",right) 
           
           
            # lokalna tablica , do której ściagane są dane od sąsiadów
            old = Array{Int64}(length(I[1])+2, length(I[2])+2)
     
            old[1      , 1      ] = d[top , left]   # left side
            old[2:end-1, 1      ] = d[I[1], left]
            old[end    , 1      ] = d[bot , left]
            old[1      , 2:end-1] = d[top , I[2]]
            old[2:end-1, 2:end-1] = d[I[1], I[2]]   # middle
            old[end    , 2:end-1] = d[bot , I[2]]
            old[1      , end    ] = d[top , right]  # right side
            old[2:end-1, end    ] = d[I[1], right]
            old[end    , end    ] = d[bot , right]
          
            # zwracana tablica new to wynik przetworzenia (lokalnie) naszej części tablicy, z której skomponowana zostanie wyjściowa
            # tablica rozproszona
            new=life_rule(old)
         
         
        end
    end

In [27]:
# reguła gry w życie (uruchamiana na lokalnych danych)
@everywhere function life_rule(old)
        m, n = size(old)
        new = similar(old, m-2, n-2)
        for j = 2:n-1
            for i = 2:m-1
                nc = +(old[i-1,j-1], old[i-1,j], old[i-1,j+1],
                       old[i  ,j-1],             old[i  ,j+1],
                       old[i+1,j-1], old[i+1,j], old[i+1,j+1])
                new[i-1,j-1] = (nc == 3 || nc == 2 && old[i,j])
            end
        end
        new
    end

In [28]:
# przykładowa tablica wejściowa
DA = @DArray [((i+j)%2) for i = 1:15, j = 1:15]

15×15 DistributedArrays.DArray{Int64,2,Array{Int64,2}}:
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0
 1  0  1  0  1  0  1  0  1  0  1  0  1  0  1
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0
 1  0  1  0  1  0  1  0  1  0  1  0  1  0  1
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0
 1  0  1  0  1  0  1  0  1  0  1  0  1  0  1
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0
 1  0  1  0  1  0  1  0  1  0  1  0  1  0  1
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0
 1  0  1  0  1  0  1  0  1  0  1  0  1  0  1
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0
 1  0  1  0  1  0  1  0  1  0  1  0  1  0  1
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0
 1  0  1  0  1  0  1  0  1  0  1  0  1  0  1
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0

In [29]:
# krok gry w życie
life_step(DA)

(15, 15) [2 3 4 5 6]
	From worker 3:	Mój zakres (1:15, 4:6) mój zakres + sąsiadujące brzegi  bot=1 top=15 left=3 right=7
	From worker 5:	Mój zakres (1:15, 10:12) mój zakres + sąsiadujące brzegi  bot=1 top=15 left=9 right=13
	From worker 2:	Mój zakres (1:15, 1:3) mój zakres + sąsiadujące brzegi  bot=1 top=15 left=15 right=4
	From worker 4:	Mój zakres (1:15, 7:9) mój zakres + sąsiadujące brzegi  bot=1 top=15 left=6 right=10
	From worker 6:	Mój zakres (1:15, 13:15) mój zakres + sąsiadujące brzegi  bot=1 top=15 left=12 right=1


15×15 DistributedArrays.DArray{Int64,2,Array{Int64,2}}:
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  1
 0  1  0  1  0  1  0  1  0  1  0  1  0  1  0

### Wbudowane proste narzędzia do pomiarów wydajności w języku Julia

In [20]:
# Sprawdzanie alokacji pamięci (ilość zaalokowanej pamięci w bytes)

@allocated fft(randn(10,10))

3146785

In [21]:
# Sprawdzanie czasu obliczeń, ilości alokacji itp.
@time fft(randn(10,10))

  0.006140 seconds (197 allocations: 17.998 KiB)


10×10 Array{Complex{Float64},2}:
 -1.73026+0.0im       -1.29945+11.6653im   …  -1.29945-11.6653im 
    9.651+11.4318im   -2.78856-0.177534im     -9.51593+2.33425im 
 -2.76109+4.1445im    -8.89496-4.87855im       19.5658-14.8469im 
 -11.6798-0.201726im  -3.39312+12.6561im       7.34868+5.1811im  
  3.51692-13.4765im    4.42437+1.58124im       6.75733-6.96393im 
 -6.01462+0.0im       -10.1085-18.9162im   …  -10.1085+18.9162im 
  3.51692+13.4765im    6.75733+6.96393im       4.42437-1.58124im 
 -11.6798+0.201726im   7.34868-5.1811im       -3.39312-12.6561im 
 -2.76109-4.1445im     19.5658+14.8469im      -8.89496+4.87855im 
    9.651-11.4318im   -9.51593-2.33425im      -2.78856+0.177534im

In [22]:
@timev fft(randn(10,10))

  0.000109 seconds (67 allocations: 8.141 KiB)
elapsed time (ns): 108604
bytes allocated:   8336
pool allocs:       67


10×10 Array{Complex{Float64},2}:
 -18.7281+0.0im       -1.46318-5.43352im  …  -1.46318+5.43352im
  4.59141+0.137518im   7.16324-8.2704im      -12.5081-8.19909im
  3.81127+4.90334im   -1.09025+5.99689im     -8.92965-5.24035im
  7.31994-11.6395im   -9.39664+7.58612im       -2.425-3.32655im
 -11.3951-8.2752im     4.87716-1.08517im     -8.99265+6.27504im
  14.8462+0.0im        8.92873-1.67702im  …   8.92873+1.67702im
 -11.3951+8.2752im    -8.99265-6.27504im      4.87716+1.08517im
  7.31994+11.6395im     -2.425+3.32655im     -9.39664-7.58612im
  3.81127-4.90334im   -8.92965+5.24035im     -1.09025-5.99689im
  4.59141-0.137518im  -12.5081+8.19909im      7.16324+8.2704im 

In [23]:
#jeśli chcemy dostać wynik pomiaru, jako wartość:
@elapsed(fft(randn(10,10)))

0.000117804

Ważne ! za pierwszym razem wykonanie funkcji trwa dluzej (kompilacja), miarodajne jest więc drugie wykonanie funkcji

dla zainteresowanych: pakiet BenchmarkTools https://github.com/JuliaCI/BenchmarkTools.jl

### Zadania

1. Zaimplementować rozwiązanie zadania z laboratorium 1 (Dekompozycja domenowa w zrównoleglaniu rozwiązywania równań różniczkowych cząstkowych) za pomocą Distiributed Arrays w Julii, przetestować implementacje  na klastrze Zeus, na wielordzeniowym węźle, zmierzyć przyspieszenie i efektywność dla zwiększającej się liczby procesów. Narysować wykresy. 

2. Zadanie dla chętnych (zrobienie go zwalnia z ostatniego zadania z przedmiotu) - uruchomić zadanie na wielu węzłach zeusa poprzez SSH cluster managera  https://docs.julialang.org/en/stable/manual/parallel-computing#ClusterManagers-1